对于只有观测变量的问题，直接根据样本值通过极大似然估计的方法求解分布的参数。

但是对于含有隐变量的问题，则要通过EM算法来逼近分布的参数。

## 1. EM算法

### 1.1 问题描述

输入：观测变量数据$Y$，隐变量数据$Z$，联合分布$P(Y,Z|\theta)$，条件分布$P(Z|Y,\theta)$

输出：模型参数$\theta$

举个例子，上面的各个变量和概率，用这个抛硬币的例子来说明。

假设有3枚硬币，分别记做A, B, C。这些硬币出现正面的概率分别是$\pi$，$p$和$q$。进行以下实验：先抛硬币A，根据结果来选择硬币B或是C。假如A抛得正面，则选择硬币B，否则选择硬币C。根据选择的硬币进行抛掷，如果为正面，则记为1，反面记为0。独立重复n次实验。

假设只能看到第二阶段抛硬币（即B或C）的结果，而不能观测抛硬币的过程（选B还是C）。则在整个抛硬币过程结束后，观测到硬币正面的概率。
$$
P(y|\theta) = \sum_{z}{P(y,z|\theta)}=\sum_{z}{P(z|\theta)P(y|z,\theta)}
$$

$P(y|\theta)$表示给定分布的参数$\theta$，硬币出现向上的概率。

$\sum_{z}{P(y,z|\theta)}$：${P(y,z|\theta)}$是$y$和$z$的联合概率。表示给定分布的参数$\theta$，同时出现隐变量$z$和观测变量$y$的概率。（例子中表示具体选择了某一个硬币，同时利用这个硬币进行抛掷，得到正面朝上的概率），对所有的情况对应的概率求和

$\sum_{z}{P(z|\theta)P(y|z,\theta)}$：给定分布的参数$\theta$，出现隐变量$z$的概率为$P(z|\theta)$，以这个概率为权重，对$P(y|z,\theta)$进行加权，$P(y|z,\theta)$的含义是给定分布的参数$\theta$和隐变量$z$具体是什么，此时硬币向上的概率是多少。

把$\sum_{z}{P(z|\theta)}$展开可以分成$\pi$和$1-\pi$，然后根据选择的硬币遵循$p$还是$q$，把$P(y|z,\theta)$分成了$p^y(1-p)^{1-y}$和$q^y(1-q)^{1-y}$，所以$P(y|\theta)$可以表示为
$$
P(y|\theta) = \pi p^y(1-p)^{1-y} + (1 - \pi)q^y(1-q)^{1-y}
$$
其中$y$是观测变量，表示某一次独立的试验观测到的结果是1或0。随机变量$z$是隐变量，即抛硬币A的结果，无法进行观测。参数$\theta=(\pi, p, q)$是模型模拟这个抛硬币过程的参数。

如果独立进行n次上述抛硬币过程，用$Y=(Y^{(1)}, Y^{(2)}, \cdots, Y^{(n)})^T$表示观测到的数据结果，用$Y=(Z^{(1)}, Z^{(2)}, \cdots, Z^{(n)})^T$表示未观测数据的结果，则观测数据的似然函数可以写成
$$
\begin{aligned}
P(Y|\theta) &= \sum_{Z}{P(Z|\theta)P(Y|Z,\theta)} \\
&= \prod_{j=1}^n{[\pi p^{y^{(j)}}(1-p)^{1-y^{(j)}} + (1-\pi) q^{y^{(j)}}(1-q)^{1-y^{(j)}}]}
\end{aligned}
$$

通过极大似然估计，求解该模型的参数$\theta=(\pi, p, q)$，即

$$
\hat{\theta} = \arg{\max_{\theta}{\log{P(Y|\theta)}}}
$$
一般求解极大似然估计的问题，是采用对参数求偏导数，令其等于0的方式求解，而这里的问题无法通过这种解析的方法来求解，因此采用EM算法。

### 1.2 EM算法步骤
初始化模型的参数，用$\theta_0 = (\pi_0, p_0, q_0)$表示。利用EM算法对参数进行迭代更新，第$i$次迭代后得到的参数记做$\theta_i = (\pi_i, p_i, q_i)$。在进行第$i+1$次迭代时，过程如下。

**E step**:

计算在模型参数$\theta_i = (\pi_i, p_i, q_i)$控制下，独立第$j$次实验观测的$y^{(j)}$是来自抛硬币B的概率是
$$
\mu_{i+1} = \frac{\pi_i(p_i)^{y^{(j)}}(1-p_i)^{1-y^{(j)}}}{\pi_i(p_i)^{y^{(j)}}(1-p_i)^{1-y^{(j)}} + (1-\pi_i)(q_i)^{y^{(j)}}(1-q_i)^{1-y^{(j)}}}
$$

**M step**
更新模型参数
$$
\begin{aligned}
&\pi_{i+1}= \frac{1}{n}\sum_{j=1}^n{\mu_{i+1}^{(j)}} \\
& p_{i+1} = \frac{\sum_{j=1}^n{u_{i+1}^{(j)}y^{(j)}}}{\sum_{j=1}^n{u_{i+1}^{(j)}}} \\
& q_{i+1} = \frac{\sum_{j=1}^n{(1-u_{i+1}^{(j)})y^{(j)}}}{\sum_{j=1}^n{(1-u_{i+1}^{(j)})}}
\end{aligned}
$$

第一个式子中，把独立的n次试验，得到的实验观测的$y^{(j)}$是来自抛硬币B的概率的平均值，则为抛硬币A后会选择硬币B的概率$\pi$。

第二个式子中，这是一个条件概率，在n次试验中，用每次会选择硬币B的概率$\mu_{i+1}^{(j)}$去对硬币B会抛出正面的结果进行加权，然后再对权重进行规范化。

第三个式子描述的内容和第二个式子差不多，区别在于选择的是硬币C来抛，选择硬币C的概率就是$1 - \mu_{i+1}^{(j)}$

重复以上的E step和M step直到收敛，即为EM算法的过程。